In [1]:
from qtpg.team import Team
from qtpg.learner import Learner
from qtpg.program import Program
from qtpg.rule import Rule
from qtpg.figure13 import Figure13
from qtpg.search_manager import SearchManager
import numpy as np
import uuid
import copy

In [2]:
# env params
memorySize = 20
legalMove = 0.1
illegalMove = -0.01
outOfBounds = -0.01
memoryRepeat = 0 #-0.01
goalReached = 1000

env = Figure13(5, 5, (2, 4), (0, 0), memorySize, legalMove, illegalMove, outOfBounds, memoryRepeat, goalReached)

In [3]:
adam_and_eve = []
env.reset()
init_team = Team(uuid.uuid4(), 0, 0, 0, 0, 0)
# init_rule = Rule(uuid.uuid4(), [0, 0, 0, 0], 0, 0)
# init_learner = Learner(uuid.uuid4(), init_rule)
# (init_team.learners).append(init_learner)
# init_team.mostRecent = init_learner
adam_and_eve.append(init_team)

for adam in adam_and_eve:
    print(adam.learners)
    print('------')
    adam.init_search(env, 2)
    print(adam.learners)
    for learner in adam.learners:
        print(learner.program.rule.action_set)
        print(learner.program.rule.region)

[]
------
2
[0, 0, 0, 4]


In [6]:
numGens = 50
maxTeamPool = 5
# init search manager and starting teams
searchManager = SearchManager(maxTeamPool)

# init_team = Team(uuid.uuid4(), 0, 0, 0, 0, 0)
# init_rule = Rule(uuid.uuid4(), [0, 0, 0, 0], 0, 0)
# init_learner = Learner(uuid.uuid4(), init_rule)
# (init_team.learners).append(init_learner)
# init_team.mostRecent = init_learner
# (searchManager.teamPool).append(init_team)

# init teams
adam_and_eve = []
env.reset()
init_team = Team(uuid.uuid4(), 0, 0, 0, 0, 0)
init_team.init_search(env, 2)
adam_and_eve.append(init_team)

# print('-----init pop start-----')
# for team in searchManager.teamPool:
#     print(f'Team id: {team.id}')
#     print('Learners:')
#     for learner in team.learners:
#         print(learner.id)
# print('-----init pop end-----')

for gen in range(numGens):
    toEvaluateAfterGen = []
    print(f'>>>>>>>>>>>>>>>>>>>>>>>Gen {gen+1} start!>>>>>>>>>>>>>>>>>>>>>>>')
    # get it started
    if gen == 0:
        for adam in adam_and_eve:
            win = adam.search(env)
            data = [adam, win]
            searchManager.evaluate_team(data)
    else:
        for parent in searchManager.teamPool:
            # only run if the team has not yet won
            if parent not in searchManager.winners:
                print(f'Parent: {parent.id}')
                env.current_state = parent.start_state

                # make offspring from parent, then let that offspring make a new rule
                child = copy.deepcopy(parent)
                child.id = uuid.uuid4()
                win = child.search(env)
                print(f'Child: {child.id}')
                print('Start Child Regions --------')
                for learner in child.learners:
                    print(learner.program.rule.region)
                print('End Child Regions --------')
                print(f'Child Fitness: {child.fitness}') 
                toEvaluateAfterGen.append([child, win])
            else:
                print(f'Skipping Team: {parent.id}')
    for data in toEvaluateAfterGen:
        searchManager.evaluate_team(data)
    print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print(f'End of Gen {gen+1} Regions')
    for i in range(len(searchManager.teamPool)):
        print(f'Team: {i+1} --> {searchManager.teamPool[i].fitness}')
        for learner in searchManager.teamPool[i].learners:
            print(learner.program.rule.region)
    print(f'>>>>>>>>>>>>>>>>>>>>>>>Gen {gen+1} end!>>>>>>>>>>>>>>>>>>>>>>>')
for team in searchManager.winners:
    print('$$$$$$$$$$$$$ winnnnnner start $$$$$$$$$$$$$$$$$$$$')
    print(team.fitness)
    print('Learners:')
    for learner in team.learners:
        print(f'Region: {learner.program.rule.region} --> Action: {learner.program.rule.action_set}')
    print('$$$$$$$$$$$$$ winnnnnner end $$$$$$$$$$$$$$$$$$$$')

>>>>>>>>>>>>>>>>>>>>>>>Gen 1 start!>>>>>>>>>>>>>>>>>>>>>>>
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
End of Gen 1 Regions
Team: 1 --> 0.5
[0, 0, 0, 4]
[1, 0, 0, 0]
>>>>>>>>>>>>>>>>>>>>>>>Gen 1 end!>>>>>>>>>>>>>>>>>>>>>>>
>>>>>>>>>>>>>>>>>>>>>>>Gen 2 start!>>>>>>>>>>>>>>>>>>>>>>>
Parent: 72090444-82bf-4656-984f-70f443fa6847
Child: c459b7a3-2972-4d09-8aad-39ebf4430b48
Start Child Regions --------
[0, 0, 0, 4]
[1, 0, 0, 0]
[0, 1, 0, 0]
End Child Regions --------
Child Fitness: 0.5
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
End of Gen 2 Regions
Team: 1 --> 0.5
[0, 0, 0, 4]
[1, 0, 0, 0]
Team: 2 --> 0.5
[0, 0, 0, 4]
[1, 0, 0, 0]
[0, 1, 0, 0]
>>>>>>>>>>>>>>>>>>>>>>>Gen 2 end!>>>>>>>>>>>>>>>>>>>>>>>
>>>>>>>>>>>>>>>>>>>>>>>Gen 3 start!>>>>>>>>>>>>>>>>>>>>>>>
Parent: 72090444-82bf-4656-984f-70f443fa6847
Child: f639e17e-eae7-407a-aeb2-c9b44297d83e
Start Child Regions --------
[0, 0, 0, 4]
[1, 0, 0, 0]
[0, 1, 0, 0]
End Child Regions --------
Child Fitne

[0, 4, 0, 3]
End Child Regions --------
Child Fitness: 2.5500000000000003
Parent: 77b2a529-9ec5-4729-9e8b-646e7dd518e8
dud
Child: b74b4cc6-d1a5-4885-8ce1-7685ea1f7264
Start Child Regions --------
[0, 0, 0, 4]
[1, 0, 0, 0]
[0, 1, 0, 1]
[1, 2, 1, 1]
[0, 0, 2, 2]
[1, 2, 0, 3]
[0, 4, 2, 0]
[1, 1, 4, 4]
[0, 4, 1, 0]
[1, 0, 2, 4]
[0, 1, 0, 0]
[1, 0, 1, 1]
End Child Regions --------
Child Fitness: 1.85
Parent: 30d82c84-1a3f-49ef-b43f-3b4be49288f6
Backtrackregioncehck: [0, 4, 0, 3]
Backtrackregioncehck: [0, 4, 0, 2]
Backtrackregioncehck: [0, 4, 0, 1]
Child: ed80618b-e743-43c7-b8db-6d5e0e494068
Start Child Regions --------
[0, 0, 0, 4]
[1, 0, 0, 0]
[0, 1, 0, 1]
[1, 2, 1, 1]
[0, 0, 2, 2]
[1, 2, 0, 3]
[0, 4, 2, 0]
[1, 1, 4, 4]
[0, 4, 1, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
End Child Regions --------
Child Fitness: 2.1500000000000004
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
End of Gen 16 Regions
Team: 1 --> 2.24
[0, 0, 0, 4]
[1, 0, 0, 0]
[0, 1, 0, 1]
[1, 2, 1, 1]
[0, 0

[0, 4, 1, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 3]
Team: 2 --> 2.5500000000000003
[0, 0, 0, 4]
[1, 0, 0, 0]
[0, 1, 0, 1]
[1, 2, 1, 1]
[0, 0, 2, 2]
[1, 2, 0, 3]
[0, 4, 2, 0]
[1, 1, 4, 4]
[0, 4, 1, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 3]
Team: 3 --> 2.4600000000000004
[0, 0, 0, 4]
[1, 0, 0, 0]
[0, 1, 0, 1]
[1, 2, 1, 1]
[0, 0, 2, 2]
[1, 2, 0, 3]
[0, 4, 2, 0]
[1, 1, 4, 4]
[0, 4, 1, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
Team: 4 --> 2.5500000000000003
[0, 0, 0, 4]
[1, 0, 0, 0]
[0, 1, 0, 1]
[1, 2, 1, 1]
[0, 0, 2, 2]
[1, 2, 0, 3]
[0, 4, 2, 0]
[1, 1, 4, 4]
[0, 4, 1, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 3]
Team: 5 --> 2.4600000000000004
[0, 0, 0, 4]
[1, 0, 0, 0]
[0, 1, 0, 1]
[1, 2, 1, 1]
[0, 0, 2, 2]
[1, 2, 0, 3]
[0, 4, 2, 0]
[1, 1, 4, 4]
[0, 4, 1, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
>>>>>>>>>>>>>>>>>>>>>>>Gen 21 end!>>>>>>>>>>>>>>>>>>>>>>>
>>>>>>>>>>>>>>>>>>>>>>>Ge

Backtrackregioncehck: [0, 4, 0, 3]
Backtrackregioncehck: [0, 4, 0, 2]
Backtrackregioncehck: [0, 4, 0, 1]
Child: bd237f86-c9fb-46ad-b6ba-dd84de5ad430
Start Child Regions --------
[0, 0, 0, 4]
[1, 0, 0, 0]
[0, 1, 0, 1]
[1, 2, 1, 1]
[0, 0, 2, 2]
[1, 2, 0, 3]
[0, 4, 2, 0]
[1, 1, 4, 4]
[0, 4, 1, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
End Child Regions --------
Child Fitness: 3.0800000000000005
Parent: 9bdc952e-4166-4cf6-b8ea-7489f30e9d85
Backtrackregioncehck: [1, 0, 0, 4]
Backtrackregioncehck: [1, 0, 1, 4]
Backtrackregioncehck: [1, 0, 1, 4]
Backtrackregioncehck: [1, 0, 1, 4]
Backtrackregioncehck: [1, 0, 1, 4]
Backtrackregioncehck: [1, 0, 1, 4]
Backtrackregioncehck: [1, 0, 1, 4]
Backtrackregioncehck: [1, 0, 1, 4]
Child: 459f5e84-a0cf-4af3-a7aa-ba026e839173
Start Child Regions --------
[0, 0, 0, 4]
[1, 0, 0, 0]
[0, 1, 0, 1]
[1, 2, 1, 1]
[0, 0, 2, 2]
[1, 2, 0, 3]
[0, 4, 2, 0]
[1, 1, 4, 4]
[0, 4, 1, 0]
[1, 0, 4, 4]

[0, 4, 1, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
End Child Regions --------
Child Fitness: 3.3900000000000006
Parent: 14b7758e-0632-4a8d-950f-3334714590c0
Backtrackregioncehck: [1, 0, 0, 4]
Backtrackregioncehck: [1, 0, 1, 4]
Backtrackregioncehck: [1, 0, 1, 4]
Backtrackregioncehck: [1, 0, 1, 4]
Backtrackregioncehck: [1, 0, 1, 4]
Backtrackregioncehck: [1, 0, 1, 4]
Backtrackregioncehck: [1, 0, 1, 4]
Backtrackregioncehck: [1, 0, 1, 4]
Child: 9432696e-c277-4403-8d44-145c3607b2f3
Start Child Regions --------
[0, 0, 0, 4]
[1, 0, 0, 0]
[0, 1, 0, 1]
[1, 2, 1, 1]
[0, 0, 2, 2]
[1, 2, 0, 3]
[0, 4, 2, 0]
[1, 1, 4, 4]
[0, 4, 1, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 2, 4]
[0, 1, 0, 0]
End Child Regions --------
Child Fitness: 3.3000000000000007
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

...

Backtrackregioncehck: [0, 4, 0, 3]
Backtrackregioncehck: [0, 4, 0, 2]
Backtrackregioncehck: [0, 4, 0, 1]
Child: 8977c23b-74ed-4968-b1f9-a9edc78f8636
Start Child Regions --------
[0, 0, 0, 4]
[1, 0, 0, 0]
[0, 1, 0, 1]
[1, 2, 1, 1]
[0, 0, 2, 2]
[1, 2, 0, 3]
[0, 4, 2, 0]
[1, 1, 4, 4]
[0, 4, 1, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
End Child Regions --------
Child Fitness: 4.320000000000001
Parent: 499c5ac7-d264-4877-bfd0-62e1711f860c
win!
Child: fbad385b-c006-40b4-a428-adb7abf666c5
Start Child Regions --------
[0, 0, 0, 4]
[1, 0, 0, 0]
[0, 1, 0, 1]
[1, 2, 1, 1]
[0, 0, 2, 2]
[1, 2, 0, 3]
[0, 4, 2, 0]
[1, 1, 4, 4]
[0, 4, 1, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1

[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 3]
>>>>>>>>>>>>>>>>>>>>>>>Gen 44 end!>>>>>>>>>>>>>>>>>>>>>>>
>>>>>>>>>>>>>>>>>>>>>>>Gen 45 start!>>>>>>>>>>>>>>>>>>>>>>>
Parent: a0fcbdf0-073b-4b16-8fdd-99fca94c1ee4
win!
Child: b2d51ab3-fc06-46d2-9e07-59f097a8bdad
Start Child Regions --------
[0, 0, 0, 4]
[1, 0, 0, 0]
[0, 1, 0, 1]
[1, 2, 1, 1]
[0, 0, 2, 2]
[1, 2, 0, 3]
[0, 4, 2, 0]
[1, 1, 4, 4]
[0, 4, 1, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 3]
[1, 4, 2, 4]
End Child Regions --------
Child Fitness: 1004.61
Parent: 6ae451e3-a5a3-4044-bcbd-9a7530775318
Backtrackregioncehck: [0, 4, 0, 3]
Backtrackregioncehck: [0, 4, 0, 2]
Backtrackregioncehck: [0, 4, 0, 1]
Child: fd80d201-e60f-4fa7-89ca-b8ea2aef2269
Start Child Regions --------
[0, 0, 0, 4]
[1, 0, 0, 0]
[0, 1, 0, 1]
[1, 2, 1, 1]
[0, 0

[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
Team: 4 --> 4.7200000000000015
[0, 0, 0, 4]
[1, 0, 0, 0]
[0, 1, 0, 1]
[1, 2, 1, 1]
[0, 0, 2, 2]
[1, 2, 0, 3]
[0, 4, 2, 0]
[1, 1, 4, 4]
[0, 4, 1, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 3]
Team: 5 --> 4.7200000000000015
[0, 0, 0, 4]
[1, 0, 0, 0]
[0, 1, 0, 1]
[1, 2, 1, 1]
[0, 0, 2, 2]
[1, 2, 0, 3]
[0, 4, 2, 0]
[1, 1, 4, 4]
[0, 4, 1, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 3]
>>>>>>>>>>>>>>>>>>>>>>>Gen 47 end!>>>>>>>>>>>>>>>>>>>>>>>
>>>>>>>>>>>>>>>>>>>>>>>Gen 48 start!>>>>>>>>>>>>>>>>>>>>>>>
Parent: 58456d12-2252-444d-a403-75c9b8840574
win!
Child: 9829e8b1-

[1, 0, 4, 4]
[0, 4, 0, 3]
Team: 3 --> 5.3400000000000025
[0, 0, 0, 4]
[1, 0, 0, 0]
[0, 1, 0, 1]
[1, 2, 1, 1]
[0, 0, 2, 2]
[1, 2, 0, 3]
[0, 4, 2, 0]
[1, 1, 4, 4]
[0, 4, 1, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 3]
Team: 4 --> 5.030000000000002
[0, 0, 0, 4]
[1, 0, 0, 0]
[0, 1, 0, 1]
[1, 2, 1, 1]
[0, 0, 2, 2]
[1, 2, 0, 3]
[0, 4, 2, 0]
[1, 1, 4, 4]
[0, 4, 1, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 0]
[1, 0, 4, 4]
[0, 4, 0, 3]
Team: 5 --> 5.3400000000000025
[0, 0, 0, 4]
[1, 0, 0, 0]
[0, 1, 0, 1]
[1, 2, 1, 1]
[0, 0, 2, 2]
[1, 2, 0, 3]
[0, 4, 2, 0]
[1, 1, 4, 4

Region: [1, 0, 4, 4] --> Action: 0
Region: [0, 4, 0, 0] --> Action: 2
Region: [1, 0, 4, 4] --> Action: 0
Region: [0, 4, 0, 0] --> Action: 2
Region: [1, 0, 4, 4] --> Action: 0
Region: [0, 4, 0, 0] --> Action: 2
Region: [1, 0, 4, 4] --> Action: 0
Region: [0, 4, 0, 3] --> Action: 2
Region: [1, 4, 2, 4] --> Action: 1
$$$$$$$$$$$$$ winnnnnner end $$$$$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$ winnnnnner start $$$$$$$$$$$$$$$$$$$$
1004.61
Learners:
Region: [0, 0, 0, 4] --> Action: 2
Region: [1, 0, 0, 0] --> Action: 0
Region: [0, 1, 0, 1] --> Action: 2
Region: [1, 2, 1, 1] --> Action: 1
Region: [0, 0, 2, 2] --> Action: 2
Region: [1, 2, 0, 3] --> Action: 0
Region: [0, 4, 2, 0] --> Action: 2
Region: [1, 1, 4, 4] --> Action: 0
Region: [0, 4, 1, 0] --> Action: 2
Region: [1, 0, 4, 4] --> Action: 0
Region: [0, 4, 0, 0] --> Action: 2
Region: [1, 0, 4, 4] --> Action: 0
Region: [0, 4, 0, 0] --> Action: 2
Region: [1, 0, 4, 4] --> Action: 0
Region: [0, 4, 0, 0] --> Action: 2
Region: [1, 0, 4, 4] --> Action: 0
Regio

In [5]:
for team in searchManager.winners:
    print('$$$$$$$$$$$$$ winnnnnner start $$$$$$$$$$$$$$$$$$$$')
    print(team.fitness)
    print('Learners:')
    for learner in team.learners:
        print(f'Region: {learner.program.rule.region} --> Action: {learner.program.rule.action_set}')
    print('$$$$$$$$$$$$$ winnnnnner end $$$$$$$$$$$$$$$$$$$$')

$$$$$$$$$$$$$ winnnnnner start $$$$$$$$$$$$$$$$$$$$
1003.13
Learners:
Region: [0, 0, 4, 4] --> Action: 2
Region: [1, 4, 0, 0] --> Action: 1
Region: [0, 0, 1, 4] --> Action: 3
Region: [1, 0, 0, 0] --> Action: 1
Region: [0, 0, 0, 3] --> Action: 2
Region: [1, 4, 0, 0] --> Action: 1
Region: [0, 0, 1, 4] --> Action: 3
Region: [1, 0, 0, 0] --> Action: 1
Region: [0, 0, 0, 3] --> Action: 2
Region: [1, 4, 0, 0] --> Action: 1
Region: [0, 0, 1, 4] --> Action: 3
Region: [1, 0, 0, 0] --> Action: 0
Region: [0, 1, 0, 1] --> Action: 2
Region: [1, 2, 1, 3] --> Action: 0
Region: [0, 4, 2, 3] --> Action: 2
Region: [1, 4, 2, 4] --> Action: 1
$$$$$$$$$$$$$ winnnnnner end $$$$$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$ winnnnnner start $$$$$$$$$$$$$$$$$$$$
1003.13
Learners:
Region: [0, 0, 4, 4] --> Action: 2
Region: [1, 4, 0, 0] --> Action: 1
Region: [0, 0, 1, 4] --> Action: 3
Region: [1, 0, 0, 0] --> Action: 1
Region: [0, 0, 0, 3] --> Action: 2
Region: [1, 4, 0, 0] --> Action: 1
Region: [0, 0, 1, 4] --> Action: 3
Regio

Region: [1, 0, 4, 4] --> Action: 0
Region: [0, 4, 0, 3] --> Action: 2
Region: [1, 4, 2, 4] --> Action: 1
$$$$$$$$$$$$$ winnnnnner end $$$$$$$$$$$$$$$$$$$$
$$$$$$$$$$$$$ winnnnnner start $$$$$$$$$$$$$$$$$$$$
1007.0100000000001
Learners:
Region: [0, 0, 4, 4] --> Action: 2
Region: [1, 4, 0, 0] --> Action: 1
Region: [0, 0, 1, 4] --> Action: 3
Region: [1, 0, 0, 0] --> Action: 1
Region: [0, 0, 0, 3] --> Action: 2
Region: [1, 4, 0, 0] --> Action: 1
Region: [0, 0, 1, 4] --> Action: 3
Region: [1, 0, 0, 0] --> Action: 1
Region: [0, 0, 0, 3] --> Action: 2
Region: [1, 4, 0, 0] --> Action: 1
Region: [0, 0, 1, 4] --> Action: 3
Region: [1, 0, 0, 0] --> Action: 0
Region: [0, 1, 0, 1] --> Action: 2
Region: [1, 2, 1, 1] --> Action: 1
Region: [0, 0, 2, 3] --> Action: 2
Region: [1, 4, 0, 0] --> Action: 1
Region: [0, 0, 1, 4] --> Action: 3
Region: [1, 0, 0, 0] --> Action: 0
Region: [0, 1, 0, 1] --> Action: 2
Region: [1, 2, 1, 3] --> Action: 0
Region: [0, 4, 1, 2] --> Action: 3
Region: [1, 0, 4, 4] --> Acti